# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [2]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [3]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'
movie_id_list = []
resp_b = BeautifulSoup(resp.text)
reg =re.compile("\d*")
movie_id_list = [{'movie_id':m["value"],'moive_name':m['data-name']} for  m in resp_b.find("select",id="sbox_mid",attrs={'name':'movie_id'}).findChildren("option",attrs={'data-name':re.compile(".*")}) if  re.search(reg,m['value'])]
print(movie_id_list)

[{'movie_id': '10545', 'moive_name': '導演先生的完美假期'}, {'movie_id': '10513', 'moive_name': '大地蜜語'}, {'movie_id': '10506', 'moive_name': '狂飆一夢'}, {'movie_id': '10491', 'moive_name': '親愛的莎瑪'}, {'movie_id': '10478', 'moive_name': '大畫特務'}, {'movie_id': '10472', 'moive_name': '戰慄糖果屋'}, {'movie_id': '10465', 'moive_name': '失戀殺人：三角禁戀'}, {'movie_id': '10440', 'moive_name': '明明會說話'}, {'movie_id': '10396', 'moive_name': '艾瑪'}, {'movie_id': '10349', 'moive_name': '罪樂園'}, {'movie_id': '10527', 'moive_name': '寄生上流：黑白版'}, {'movie_id': '10490', 'moive_name': '千日千夜'}, {'movie_id': '10464', 'moive_name': '咒怨2 電影版'}, {'movie_id': '10458', 'moive_name': '鳴鳥不飛：烏雲密布'}, {'movie_id': '10438', 'moive_name': '哥布林殺手 劇場版'}, {'movie_id': '10437', 'moive_name': '女鬼橋'}, {'movie_id': '10421', 'moive_name': '怨咒'}, {'movie_id': '10401', 'moive_name': '娘娘腔日記'}, {'movie_id': '10388', 'moive_name': '汪星人的奇幻漂流'}, {'movie_id': '10383', 'moive_name': '極地守護犬'}, {'movie_id': '10371', 'moive_name': '叔．叔'}, {'movie_id': '10359', 'mo

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [4]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = "10349"

In [5]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
parameter = {'movie_id':movie_id}
header = {
'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb'
}
resp_areas = requests.get(url,params=parameter,headers=header)
print([{'area_name':d['title'],'area_id':d['area_id']} for d in resp_areas.json()])

[{'area_name': '台北市', 'area_id': 28}, {'area_name': '新北市', 'area_id': 8}, {'area_name': '桃園', 'area_id': 16}, {'area_name': '新竹', 'area_id': 20}, {'area_name': '台中', 'area_id': 2}, {'area_name': '台南', 'area_id': 10}, {'area_name': '高雄', 'area_id': 17}, {'area_name': '宜蘭', 'area_id': 11}]


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [6]:
# 指定放映地區
area_id="28"

In [22]:
# 向網站發送請求
params = {'movie_id':movie_id,'area_id':area_id}
url = f"https://movies.yahoo.com.tw/movietime_result.html"
resp = requests.get(url,params=params)
resp_b = BeautifulSoup(resp.text)
print([{'month':d.find('p',class_='month').text,'day':d.find('h3',class_='day').text,'week':d.find('p',class_='week').text} for d in resp_b.find('ul',class_='date_timetable_picker').find_all('li')])
# 列印播放日期


[{'month': '三月', 'day': '11', 'week': '今天'}, {'month': '三月', 'day': '12', 'week': '明天'}, {'month': '三月', 'day': '13', 'week': '禮拜五'}, {'month': '三月', 'day': '14', 'week': '禮拜六'}, {'month': '三月', 'day': '15', 'week': '禮拜天'}]


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [30]:
# 選定要觀看的日期
date='2020-03-11'

In [65]:
# 向網站發送請求，獲取上映的電影院及時間資訊
params = {'movie_id':movie_id,'area_id':area_id,'date':date}
url = f"https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
resp = requests.get(url,params=params)
resp_b = BeautifulSoup(resp.json()['view'])

print(resp.json()['view'])  # 若有需要，列印出json原始碼



[{'address': '國賓影城(台北長春廣場)', 'phone': '02-25155755', 'taps': ['數位'], 'times': ['15:00', '19:25', '21:50']}, {'address': '喜滿客絕色影城', 'phone': '02-23811339', 'taps': ['數位'], 'times': ['19:00', '23:10']}, {'address': '台北信義威秀影城', 'phone': '02-87805566', 'taps': ['數位'], 'times': ['20:35', '23:00']}, {'address': '百老匯數位影城', 'phone': '02-86636128', 'taps': ['數位'], 'times': ['09:50', '12:10', '14:30', '16:50', '19:10', '21:30', '23:50']}, {'address': '今日秀泰影城', 'phone': '02-23751669', 'taps': ['數位'], 'times': ['11:00', '15:40', '20:20', '22:50']}, {'address': '誠品電影院', 'phone': '02-66365888', 'taps': ['數位'], 'times': ['14:00', '20:55']}, {'address': '喜樂時代影城南港店', 'phone': '02-27888185', 'taps': ['數位'], 'times': ['20:10']}]


In [10]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''


'\n\n  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表\n  \n  Your code here.\n\n'